In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
import scipy.io
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,roc_auc_score
import matplotlib.pyplot as plt
import scipy
from scipy.signal import find_peaks
from scipy.signal import medfilt, butter, filtfilt
import pywt


# Import Data and concat into dataframe

In [54]:
def load_ecg_data(mat_filename):
    # Load MATLAB file
    mat_data = scipy.io.loadmat(mat_filename)
    # Assume the ECG data is stored under the key 'ecg_data'
    ecg_data = mat_data['val'].ravel()
    return ecg_data

# Specify the directory where your data files are located
data_dir = '/content/drive/My Drive/PHIC A-Fib Project/training2017/'

In [55]:
t = load_ecg_data('/content/drive/My Drive/PHIC A-Fib Project/training2017/A00007.mat')
t.shape[0]

9000

In [91]:
data_list = []
for i in range(1,4000):
    mat_file_name = f'A{str(i).zfill(5)}.mat'  # Construct the file name
    data =  load_ecg_data(os.path.join(data_dir, mat_file_name))
    if data.shape[0] > 9000:
      data_list.append(data[0:9000])
    else:
      data_list.append(data)
label_file_name = 'REFERENCE.csv'  # Replace with your label file name
labels = pd.read_csv(os.path.join(data_dir, label_file_name), header=None, names=['Label'])
labels = labels.reset_index(drop=True)
labels = labels.iloc[0:10,:]

ecg_data_df = pd.DataFrame(data_list)

KeyboardInterrupt: ignored

In [57]:
with open('/content/drive/My Drive/PHIC A-Fib Project/training2017/A00003.hea', 'r') as f:
    header_content1 = f.read()
    header_content1 = header_content1.split(' ')
header_content1

['A00003',
 '1',
 '300',
 '18000',
 '12:04:05',
 '1/04/2000',
 '\nA00003.mat',
 '16+24',
 '1000/mV',
 '16',
 '0',
 '56',
 '0',
 '0',
 'ECG',
 '\n']

A00001: The record name.
<br>1: Number of signals (leads).
<br>300: Sampling frequency in Hz.
<br>9000: Number of samples in the record.
<br>05:05:15 1/05/2000: Start time and date of the recording.
<br>A00001.mat: Name of the file containing the signal data.
<br>16+24: ADC resolution in bits.
<br>1000/mV: ADC gain.
<br>16: Baseline value.
<br>0 -127 0 0: Other signal-specific parameters.
<br>ECG: Signal type.

#Data Preprocess

### 1. Normalize or Standardize the Signal

In [58]:

# Normalize the data between -1 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
ecg_data_df = scaler.fit_transform(ecg_data_df)

### 2. Remove Artifacts and Noise

Filter the Dataset
</br>ECG signals often contain noise and artifacts from various sources such as electrical interference, muscle contractions, or movement of the patient. Thus, filtering aims to reduce this noise while preserving the true ECG signal as much as possible.

In [59]:
#filtering the raw signals
# Median filtering
ecg_medfilt = medfilt(ecg_data_df, kernel_size=3)


In [60]:
# Low-pass filtering
lowcut = 0.05
highcut = 20.0
nyquist = 0.5 * 300.0
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(4, [low, high], btype='band')
ecg_lowpass = filtfilt(b, a, ecg_data_df)

In [61]:
# Wavelet filtering
coeffs = pywt.wavedec(ecg_data_df, 'db4', level=1)
threshold = np.std(coeffs[-1]) * np.sqrt(2*np.log(len(ecg_data_df)))
coeffs[1:] = (pywt.threshold(i, value=threshold, mode='soft') for i in coeffs[1:])
ecg_wavelet = pywt.waverec(coeffs, 'db4')

In [62]:
#pad the signal with zeroes
def pad_data(original_data,filtered_data):
  # Calculate the difference in length between the original data and filtered data
  diff = original_data.shape[1] - filtered_data.shape[1]
    # pad the shorter array with zeroes
  if diff > 0:
          # Create an array of zeros with the same shape as the original data
      padding = np.zeros((filtered_data.shape[0], diff))
      # Concatenate the filtered data with the padding
      padded_data = np.concatenate((filtered_data, padding))
  elif diff < 0:
      padded_data = filtered_data[:,:-abs(diff)]
  elif diff == 0:
      padded_data = filtered_data
  return padded_data


### 3.Signal Quality Assessment

In [63]:
def mse(original_data, filtered_data):
    filter_data = pad_data(original_data,filtered_data)
    return np.mean((original_data - filter_data) ** 2)
# Calculate MSE
mse_value_m = mse(ecg_data_df, ecg_medfilt)
mse_value_l = mse(ecg_data_df, ecg_lowpass)
mse_value_w = mse(ecg_data_df, ecg_wavelet)
print("MSE value of Median Filtering:", mse_value_m)
print("MSE value of Low-pass Filtering:", mse_value_l)
print("MSE value of Wavelet Filtering:", mse_value_w)

MSE value of Median Filtering: 0.2835397567380967
MSE value of Low-pass Filtering: 0.059816621269229
MSE value of Wavelet Filtering: 0.0006313483777052557


Based on the MSE value displayed above, wavelet filtering is chosen.



###4. Train Test Data Split

In [68]:
def encoding_labels(df,label_col,category):
  for i in range(df.shape[0]):
    j = 0
    while j < len(category):
      if df.iloc[i][label_col] == category[j]:
        df.at[i,label_col] = float(j)
      else:
        j+=1
  df[label_col] = df[label_col].astype(float)
  return df[label_col]

In [69]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(ecg_wavelet, labels, test_size=0.2, random_state=42)
#one hot encoding of labels
category = ['N','A','O','~']
y_train = y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
y_train_encoded = encoding_labels(y_train,'Label',category)
y_test_encoded = encoding_labels(y_test,'Label',category)

In [70]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Label   8 non-null      float64
dtypes: float64(1)
memory usage: 192.0 bytes


###5. Feature Extraction

Choosing relevant features
In this context, the following features are being used:

1. T amplitude: The height of the T wave on the electrocardiogram (ECG) graph, which symbolizes the heart’s return to rest following a contraction
2.   R amplitude: The height of the R wave on the ECG graph represents the heart muscle’s initial contraction.
3.  RR interval: The time between two consecutive R waves on the ECG graph, representing the time between heartbeats.
4.  QRS duration: The time it takes for the QRS complex, representing the electrical signal traveling through the ventricles.


#### Feature Extraction of the Train Set

In [35]:
X_train.shape

(8, 9000)

In [37]:
# Initializing an empty list to store the features
features = []
# Extracting features for each sample
for i in range(X_train.shape[0]):
    #Finding the R-peaks, which store index of r_peaks
    r_peaks = scipy.signal.find_peaks(X_train[i])[0]
    #Initialize lists to hold R-peak and T-peak amplitudes
    r_amplitudes = []
    t_amplitudes = []
    # Iterate through R-peak locations to find corresponding T-peak amplitudes
    for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
        #with a window size 200, the time lag is set to be 0.67s which ensures the T-wave is captured for analysis without being cut off.
        search_radius_t = 200
        search_window_t = slice(r_peak,min(len(X_train[i]),r_peak+search_radius_t))
        search_radius_qs = 30  # 0.1* 300hz
        search_window_q = slice(max(0,r_peak - search_radius_qs),r_peak)
        search_window_s = slice(r_peak,min(len(X_train[i]),r_peak+ search_radius_qs))
        t_peak = np.argmin(X_train[i][search_window_t]) + r_peak
        q_peak = r_peak - np.argmin(X_train[i][search_window_q])
        s_peak =  np.argmin(X_train[i][search_window_s]) + r_peak
        #Append the R-peak amplitude and T-peak amplitude to the lists
        r_amplitudes.append(X_train[i][r_peak])
        t_amplitudes.append(X_train[i][t_peak])
    # extracting singular value metrics from the r_amplitudes
    std_r_amp = np.std(r_amplitudes)
    mean_r_amp = np.mean(r_amplitudes)
    median_r_amp = np.median(r_amplitudes)
    sum_r_amp = np.sum(r_amplitudes)
    # extracting singular value metrics from the t_amplitudes
    std_t_amp = np.std(t_amplitudes)
    mean_t_amp = np.mean(t_amplitudes)
    median_t_amp = np.median(t_amplitudes)
    sum_t_amp = np.sum(t_amplitudes)
    # Find the time between consecutive R-peaks
    rr_intervals = np.diff(r_peaks)
    # Calculate the time duration of the data collection
    time_duration = len(X_train[i])/300 # !!! need to check later - assuming all ecg record sampling rate is 300hz
    # set the sampling rate
    sampling_rate = 300
    # Calculate heart rate
    duration = len(X_train[i]) / sampling_rate
    heart_rate = (len(r_peaks) / duration) * 60 #heart beat per second * 60
    # QRS duration slice(strart,stop[,step])
    qrs_durations = []
    qrs_duration = (s_peak - q_peak) / sampling_rate
    qrs_durations.append(qrs_duration)
    # extracting singular value metrics from the qrs_durations
    std_qrs = np.std(qrs_duration)
    mean_qrs = np.mean(qrs_duration)
    median_qrs = np.median(qrs_duration)
    sum_qrs = np.sum(qrs_duration)
    # Extracting the singular value metrics from the RR-interval
    std_rr = np.std(rr_intervals)
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sum_rr = np.sum(rr_intervals)
    # Extracting the overall standard deviation
    std = np.std(X_train[i])
    # Extracting the overall mean
    mean = np.mean(X_train[i])
    # Appending the features to the list
    features.append([mean, std, std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr, heart_rate])
# Converting the list to a numpy array
features = np.array(features)

In [38]:
features

array([[-1.12165132e-01,  5.88447523e-01,  0.00000000e+00,
         2.00000000e-02,  2.00000000e-02,  2.00000000e-02,
         7.05963160e-01,  7.70205442e-02,  1.17844258e-01,
         6.88563665e+01,  8.48637146e-02, -1.03097218e+00,
        -1.05090816e+00, -9.21689125e+02,  8.98600000e+03,
         7.68140002e+00,  1.00627100e+01,  8.00000000e+00,
         1.78800000e+03],
       [-8.57089973e-02,  6.37402006e-01,  0.00000000e+00,
         9.66666667e-02,  9.66666667e-02,  9.66666667e-02,
         7.74806403e-01,  5.73690016e-02,  8.93063647e-02,
         5.78853226e+01,  6.85316430e-02, -1.02586294e+00,
        -1.04370281e+00, -1.03509570e+03,  8.95800000e+03,
         7.14553273e+00,  8.88690476e+00,  7.00000000e+00,
         2.01800000e+03],
       [-5.15833632e-02,  6.80053984e-01,  0.00000000e+00,
         1.03333333e-01,  1.03333333e-01,  1.03333333e-01,
         8.33286639e-01,  5.33903605e-02,  1.17732806e-01,
         6.20395989e+01,  9.23185926e-02, -1.00745499e+00,
    

###6. Feature extraction of the test set



In [39]:
features_test = []
# Extracting features for each sample
for i in range(X_test.shape[0]):
    #Finding the R-peaks, which store index of r_peaks
    r_peaks = scipy.signal.find_peaks(X_test[i])[0]
    #Initialize lists to hold R-peak and T-peak amplitudes
    r_amplitudes = []
    t_amplitudes = []
    # Iterate through R-peak locations to find corresponding T-peak amplitudes
    for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
        #with a window size 200, the time lag is set to be 0.67s which ensures the T-wave is captured for analysis without being cut off.
        search_radius_t = 200
        search_window_t = slice(r_peak,min(len(X_test[i]),r_peak+search_radius_t))
        search_radius_qs = 30  # 0.1* 300hz
        search_window_q = slice(max(0,r_peak - search_radius_qs),r_peak)
        search_window_s = slice(r_peak,min(len(X_test[i]),r_peak+ search_radius_qs))
        t_peak = np.argmin(X_test[i][search_window_t]) + r_peak
        q_peak = r_peak - np.argmin(X_test[i][search_window_q])
        s_peak =  np.argmin(X_test[i][search_window_s]) + r_peak
        #Append the R-peak amplitude and T-peak amplitude to the lists
        r_amplitudes.append(X_test[i][r_peak])
        t_amplitudes.append(X_test[i][t_peak])
    # extracting singular value metrics from the r_amplitudes
    std_r_amp = np.std(r_amplitudes)
    mean_r_amp = np.mean(r_amplitudes)
    median_r_amp = np.median(r_amplitudes)
    sum_r_amp = np.sum(r_amplitudes)
    # extracting singular value metrics from the t_amplitudes
    std_t_amp = np.std(t_amplitudes)
    mean_t_amp = np.mean(t_amplitudes)
    median_t_amp = np.median(t_amplitudes)
    sum_t_amp = np.sum(t_amplitudes)
    # Find the time between consecutive R-peaks
    rr_intervals = np.diff(r_peaks)
    # Calculate the time duration of the data collection
    time_duration = len(X_test[i])/300 # !!! need to check later - assuming all ecg record sampling rate is 300hz
    # set the sampling rate
    sampling_rate = 300
    # Calculate heart rate
    duration = len(X_test[i]) / sampling_rate
    heart_rate = (len(r_peaks) / duration) * 60 #heart beat per second * 60
    # QRS duration slice(strart,stop[,step])
    qrs_durations = []
    qrs_duration = (s_peak - q_peak) / sampling_rate
    qrs_durations.append(qrs_duration)
    # extracting singular value metrics from the qrs_durations
    std_qrs = np.std(qrs_duration)
    mean_qrs = np.mean(qrs_duration)
    median_qrs = np.median(qrs_duration)
    sum_qrs = np.sum(qrs_duration)
    # Extracting the singular value metrics from the RR-interval
    std_rr = np.std(rr_intervals)
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sum_rr = np.sum(rr_intervals)
    # Extracting the overall standard deviation
    std = np.std(X_test[i])
    # Extracting the overall mean
    mean = np.mean(X_test[i])
    # Appending the features to the list
    features_test.append([mean, std, std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr, heart_rate])
# Converting the list to a numpy array
features_test = np.array(features_test)

In [40]:
features_test

array([[-7.17282946e-02,  7.05489227e-01,  0.00000000e+00,
         9.66666667e-02,  9.66666667e-02,  9.66666667e-02,
         8.63314508e-01,  3.72975055e-03, -2.73672867e-02,
         4.53910642e+00,  6.73779807e-02, -1.03177017e+00,
        -1.03878041e+00, -1.25566430e+03,  8.95200000e+03,
         6.51559298e+00,  7.36184211e+00,  5.00000000e+00,
         2.43400000e+03],
       [-9.55513758e-02,  6.38279827e-01,  0.00000000e+00,
         9.33333333e-02,  9.33333333e-02,  9.33333333e-02,
         8.06841053e-01,  3.13876123e-02,  1.09319751e-01,
         3.22978530e+01,  9.48944246e-02, -1.00586740e+00,
        -1.02955198e+00, -1.03503755e+03,  8.99300000e+03,
         7.35043780e+00,  8.74805447e+00,  7.00000000e+00,
         2.05800000e+03]])

In [41]:
features.reshape(features.shape[0],features.shape[1],1).shape

(8, 19, 1)

# Model building and training

In [42]:
data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9],
                 [10, 11, 12],
                 [13, 14, 15],
                 [16, 17, 18]])

In [43]:
reshaped_data = data.reshape(-1, 2
, 1)
print(reshaped_data)

[[[ 1]
  [ 2]]

 [[ 3]
  [ 4]]

 [[ 5]
  [ 6]]

 [[ 7]
  [ 8]]

 [[ 9]
  [10]]

 [[11]
  [12]]

 [[13]
  [14]]

 [[15]
  [16]]

 [[17]
  [18]]]


In [44]:
features_test.dtype

dtype('float64')

In [47]:
#reshape features
features =np.asarray(features).astype('float32')
features_test =np.asarray(features_test).astype('float32')

feature_input = features.reshape(features.shape[0],features.shape[1],1)
features_test = features_test.reshape(features_test.shape[0],features_test.shape[1],1)

In [76]:
#build LSTM architecture
model = Sequential()
model.add(LSTM(64, input_shape=(features.shape[1], 1)))
model.add(Dense(4, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#train the model
history = model.fit(feature_input, y_train, validation_data=(features_test, y_test), epochs=50, batch_size=32)

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 1.2458 - accuracy: 0.1250 - val_loss: 1.2039 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 76ms/step - loss: 1.1967 - accuracy: 0.5000 - val_loss: 1.1799 - val_accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 94ms/step - loss: 1.1666 - accuracy: 0.6250 - val_loss: 1.1608 - val_accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 78ms/step - loss: 1.1394 - accuracy: 0.6250 - val_loss: 1.1552 - val_accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 84ms/step - loss: 1.1116 - accuracy: 0.6250 - val_loss: 1.1818 - val_accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 97ms/step - loss: 1.0862 - accuracy: 0.6250 - val_loss: 1.2071 - val_accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 84ms/step - loss: 1.0607 - accuracy: 0.6250 - val_loss: 1.1934 - val_accuracy: 0.5000
Epoch 8/50
1/1 [=============

In [81]:
# Make predictions on the validation set
y_pred = model.predict(features_test)
# Convert the predicted values to binary labels
y_pred = np.argmax(y_pred, axis=1)


1/1 [==============================] - 0s 147ms/step


In [82]:
y_pred

array([0, 0])

# Model Evaluation

In [90]:
# calculate the accuracy
acc = accuracy_score(y_test, y_pred)
#calculate the AUC score
auc = round(roc_auc_score(y_test, y_pred),2)
#classification report provides all metrics e.g. precision, recall, etc.
print(classification_report(y_test, y_pred))
print('auc score:',auc,'acc score:',acc)

              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67         1
         1.0       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

auc score: 0.5 acc score: 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
all_met

'              precision    recall  f1-score   support\n\n         0.0       0.50      1.00      0.67         1\n         1.0       0.00      0.00      0.00         1\n\n    accuracy                           0.50         2\n   macro avg       0.25      0.50      0.33         2\nweighted avg       0.25      0.50      0.33         2\n'